In [ ]:
import numpy as np
import os
from utils.utilities import *
from sklearn.model_selection import train_test_split
#import matplotlib.pyplot as plt
import pandas as pd
#import seaborn as sns
#%matplotlib inline

In [ ]:
X_train, labels_train, list_ch_train = read_data(data_path="../data/HAR_Dataset", split="train") # train
X_test, labels_test, list_ch_test = read_data(data_path="../data/HAR_Dataset", split="test") # test

assert list_ch_train == list_ch_test, "Mistmatch in channels!"

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import os
import csv
import tempfile
import zipfile
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Input, Dense, Activation,Reshape
from tensorflow.keras.layers import Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers
#from tensorflow.keras.backend.tensorflow_backend import set_session
from tensorflow.keras import backend as K
from utils.utilities import *
import matplotlib.pyplot as plt


np.random.seed(813306)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 忽略 Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2
#tf.logging.set_verbosity(tf.logging.ERROR)
#config = tf.ConfigProto()
#config.gpu_options.allocator_type = 'BFC'  # A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
#config.gpu_options.per_process_gpu_memory_fraction = 1
#config.gpu_options.allow_growth = True
#set_session(tf.Session(config=config))


#数据预处理
tf.keras.backend.set_image_data_format('channels_last')
#构建数据集 channel_last
#构建数据集 channel_last
def load_data():    
    X_train, labels_train, list_ch_train = read_data(data_path="../data/HAR_Dataset", split="train") # train
    X_test, labels_test, list_ch_test = read_data(data_path="../data/HAR_Dataset", split="test") # test
    assert list_ch_train == list_ch_test, "Mistmatch in channels!"
    x_train = X_train[:,:,np.newaxis,:]
    x_val = X_test[:,:,np.newaxis,:]
    y_train = to_categorical(labels_train)
    y_val = to_categorical(labels_test)
    return (x_train,y_train),(x_val,y_val)



#build the model
def build_resnet(input_shape, n_feature_maps, nb_classes, dropout):
    print('build conv_x')
    x = Input(shape=(input_shape),name = 'input')
    x_total = []
    for i in range(input_shape[-1]):
        x_temp = Reshape((input_shape[0],1,1),name = 'reshape_'+str(i+1))(x[:,:,:,i])
        
        x_temp = keras.layers.Conv2D(16, (17, 1),strides = (1,1), padding='same',name = 'conv2d_'+str(i+1)+'_1')(x_temp)
        x_temp = keras.layers.AveragePooling2D(pool_size=(3, 1), strides=None, padding='same', data_format=None,name = 'average_pooling2d_'+str(i+1)+'_1')(x_temp)
        x_temp = keras.layers.BatchNormalization(name = 'batch_normalization_'+str(i+1)+'_1')(x_temp)
        x_temp = Activation('relu',name = 'activation_'+str(i+1)+'_1')(x_temp)
        
        x_temp = keras.layers.Conv2D(32, (17, 1),strides = (1,1), padding='same',name = 'conv2d_'+str(i+1)+'_2')(x_temp)
        x_temp = keras.layers.AveragePooling2D(pool_size=(3, 1), strides=None, padding='same', data_format=None,name = 'average_pooling2d_'+str(i+1)+'_2')(x_temp)
        x_temp = keras.layers.BatchNormalization(name = 'batch_normalization_'+str(i+1)+'_2')(x_temp)
        x_temp = Activation('relu',name = 'activation_'+str(i+1)+'_2')(x_temp)
        
        x_temp = keras.layers.Conv2D(16, (17, 1),strides = (1,1), padding='same',name = 'conv2d_'+str(i+1)+'_3')(x_temp)
        x_temp = keras.layers.AveragePooling2D(pool_size=(3, 1), strides=None, padding='same', data_format=None,name = 'average_pooling2d_'+str(i+1)+'_3')(x_temp)
        x_temp = keras.layers.BatchNormalization(name = 'batch_normalization_'+str(i+1)+'_3')(x_temp)
        x_temp = Activation('relu',name = 'activation_'+str(i+1)+'_3')(x_temp)
        
        x_temp = keras.layers.Conv2D(16, (17, 1),strides = (1,1), padding='same',name = 'conv2d_'+str(i+1)+'_4')(x_temp)
        x_temp = keras.layers.AveragePooling2D(pool_size=(2, 1), strides=None, padding='same', data_format=None,name = 'average_pooling2d_'+str(i+1)+'_4')(x_temp)
        x_temp = keras.layers.BatchNormalization(name = 'batch_normalization_'+str(i+1)+'_4')(x_temp)
        x_temp = Activation('relu',name = 'activation_'+str(i+1)+'_4')(x_temp)
        
        x_temp = keras.layers.Conv2D(16, (9, 1),strides = (1,1), padding='same',name = 'conv2d_'+str(i+1)+'_5')(x_temp)
        x_temp = keras.layers.AveragePooling2D(pool_size=(2, 1), strides=None, padding='same', data_format=None,name = 'average_pooling2d_'+str(i+1)+'_5')(x_temp)
        x_temp = keras.layers.BatchNormalization(name = 'batch_normalization_'+str(i+1)+'_5')(x_temp)
        x_temp = Activation('relu',name = 'activation_'+str(i+1)+'_5')(x_temp)
        
      
        x_temp = keras.layers.Conv2D(1, (9, 1),strides = (1,1), padding='same',name = 'conv2d_'+str(i+1)+'_7')(x_temp)
        
        
        x_total.append(x_temp)

    
    x_total = K.concatenate(x_total , axis=-1)
    conv_x = keras.layers.BatchNormalization(name = 'batch_normalization_left_first')(x_total)  # 853
 
    conv_x = keras.layers.Conv2D(n_feature_maps, (16, 1), padding='same',name = 'conv2d_left_1')(conv_x)  # input size == ouput size
    conv_x = keras.layers.BatchNormalization(name = 'batch_normalization_left_1')(conv_x)
    conv_x = Activation('relu',name = 'activation_left_1')(conv_x)

    '''
    print('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps * 2, (16, 1), padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)

    conv_y = Dropout(dropout)(conv_y)
    '''
    print('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps, (8, 1), padding='same',name = 'conv2d_left_2')(conv_x)
    #conv_z = keras.layers.BatchNormalization()(conv_x)

    is_expand_channels = not (input_shape[-1] == n_feature_maps)  # 若当前输出和跨层连接的x，通道数不同，则采用1*1卷积使得通道数相同
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps, (1, 1), padding='same',name = 'conv2d_right')(x_total)
        #shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = x_total#keras.layers.BatchNormalization()(x_total)

    #print('Merging skip connection')
    # y = merge([shortcut_y, conv_z], mode='sum')
    y = keras.layers.Add(name = 'add')([shortcut_y, conv_z])
    y = keras.layers.BatchNormalization(name = 'batch_normalization_back_1')(y)
    y = Activation('relu',name = 'activation_back_1')(y)


    full = keras.layers.GlobalAveragePooling2D(name = 'global_average_pooling2d')(y)
    out = Dense(nb_classes, activation='softmax',name = 'dense')(full)
    print('        -- model was built.')
    return x, out


In [ ]:
if __name__ == '__main__':
    num = 6
    channels = 3
    dropout = 0.2
    nb_epochs = 200
    batch_size = 50
    data_row = 128
    data_column = 1
    trainpath = r'./data'

    (x_train,y_train),(x_val,y_val) = load_data()


    tf.keras.backend.set_image_data_format('channels_last')


    input_shape = (data_row, data_column,channels)

    print('train dataset size:',x_train.shape[0])
    print('validation dataset size:',x_val.shape[0])
    num_classes = y_train.shape[1]
    
    x, y = build_resnet(input_shape, 64, num, dropout)  # 建立resnet只考虑了单个example
    model = Model(inputs=x, outputs=y)
    model.summary()
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.9,
                                  patience=20, min_lr=0.00005)
    hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=nb_epochs,
                     verbose=1, validation_data=(x_val, y_val), callbacks=[reduce_lr])  # 回调函数会在训练的时候适当被调用


    # 测试
    print("------------------------ 测试中---------------------------")
    #evaluation of the model
    scores = model.evaluate(x_val,y_val)
    print('Baseline Error: %.2f%%'%(100 * (1 - scores[1])))
    keras.models.save_model(model, '../model/model_ql.h5')
    keras_file = '../model/test_resnet_v4.h5'
    _, zip1 = tempfile.mkstemp('.zip') 
    with zipfile.ZipFile(zip1, 'w', compression=zipfile.ZIP_DEFLATED) as f:
        f.write(keras_file)
    print("Size of the unpruned model before compression: %.2f Mb" % 
          (os.path.getsize(keras_file) / float(2**20)))
    print("Size of the unpruned model after compression: %.2f Mb" % 
          (os.path.getsize(zip1) / float(2**20)))